#### Festlegung für die Darstellung von interaktiven Graphiken innerhalb des Notebooks

In [ ]:
%pylab inline

#### Import der notwendigen Pakete und Module

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as const
import scipy
from scipy.io import wavfile
from pylab import *
from scipy.fftpack import fft, ifft
from IPython.display import Audio

import functions.dsvorg as do
import functions.data_resample as resample
import functions.bsfilter as bsfilter
import functions.classes as dc
import functions.tproc_new as tp

[Table of Contents](table_of_contents.ipynb)

In [ ]:
import functions.PrettyTable as pt

##### Protokoll
## Signale, akustische Ereignisse

<table align="left" width="400">
    <colgroup>
       <col span="1" style="width: 50%;">
       <col span="1" style="width: 50%;">
    </colgroup>
  <tr>
    <th>Name</th>
    <th>Vorname</th> 
  </tr>
  <tr>
    <td> Flach</td>
    <td> Gudrun</td>
  </tr>
      <tr>
    <td> Mustermann</td>
    <td> Max</td>
  </tr>

</table>


#### Hinweis:
In dem Verzeichnis `sound` sind die für das Praktikum erforderlichen Signale enthalten. Alle im Rahmen der Experimente erstellten Signale werden unter den angegebenen Namen ebenfalls in diesem Verzeichnis abgespeichert.

In [ ]:
info_data = []
head = ['Dateiname', 'Abtastrate', 'Datenformat', 'Anz. ATW', 'Maximum', 'Minimum']

### Signalinformationen
Für die korrekte Verarbeitung digitaler Signale sind eine Reihe von Informationen erforderlich, die in dem Dateiheader gehalten werden.  
### <font color="#008000">Aufgabe 1</font>
Untersuchen Sie die aufgeführten Signale und berechnen Sie die Signaldauer. 

#### <font color="#FF0000">Eingabezelle 1</font>

In [ ]:
#file = 'gaense'
#file = 'glocken'
file = 'mann'
#file = 'rauschen'
#file = 'traktor_fahrt'
#file = 'traktor_stand'
#file = 'wasser'

In [ ]:
info_data.append(do.get_info(file))
pt.PrettyTable(info_data, head)

##### Protokoll
#### Aufgabe 1 Signaldauer

<table align="left" width="400">
    <colgroup>
       <col span="1" style="width: 50%;">
       <col span="1" style="width: 50%;">
    </colgroup>
  <tr>
    <th>Datei</th>
    <th>Signaldauer in ms</th> 
  </tr>
  <tr>
    <td> gaense</td>
    <td> </td>
  </tr>
  <tr>
    <td> glocken</td>
    <td> </td>
  </tr>
  <tr>
    <td> mann</td>
    <td> </td>
  </tr>
  <tr>
    <td> rauschen</td>
    <td> </td>
  </tr>
  <tr>
    <td> traktor_fahrt</td>
    <td> </td>
  </tr>
  <tr>
    <td> traktor_stand</td>
    <td> </td>
  </tr>
  <tr>
    <td> wasser</td>
    <td> </td>
  </tr>

</table>


### Normierung
Die vorliegenden Daten unterscheiden sich hinsichtlich einiger Kriterien. Um Daten miteinander vergleichen zu können, sollten sie normiert werden. Zunächst soll eine Normierung auf den maximalen Betragswert des Signals vorgenommen werden.

### <font color="#008000">Aufgabe 2</font>
Sie sehen und hören ein nichtnormiertes und ein nichtnormiertes Signal. Vergleichen Sie die Darstellungen und die 3 Audioausgaben. Beschreiben und erklären Sie den beobachteten Effekt in der **Tabelle Normierung**!

In [ ]:
fs, data = do.load_data_o('craco_o')
fs, data_norm = do.load_data('craco_o')
t = np.arange(0, len(data)/fs, 1/fs)
plt.figure(figsize=(15,5))
subplot(221)
plt.plot(t, data)
plt.title('Signal original')
plt.xlabel('t in s')
subplot(222)
plt.plot(t, data_norm)
plt.title('Signal normiert')
plt.xlabel('t in s')
plt.savefig('a2-2.jpg')
plt.show()

#### Audioausgabe 1
Sie hören zunächst das originale und unmittelbar anschließend das normierte Signal.

In [ ]:
data_out = np.append(data, data_norm)
Audio(data = data_out, rate = fs)

#### Audioausgabe 2
Sie hören das nicht normierte Signal. 

In [ ]:
Audio(data = data, rate = fs)

#### Audioausgabe 3
Sie hören das normierte Signal.  

In [ ]:
Audio(data = data_norm, rate = fs)

##### Protokoll
#### Aufgabe 2  Normierung
<table>
    <colgroup>
       <col span="1" style="width: 12%;">
       <col span="1" style="width: 40%;">
       <col span="1" style="width: 40%;">
    </colgroup>
  <tr>
    <th>Audioausgabe</th>
    <th>Beobachtung</th> 
    <th>Erklärung</th>
  </tr>
  <tr>
    <td>nicht normiert + normiert</td>
    <td> </td>
    <td> </td>
  </tr>
  <tr>
    <td>nicht normiert</td>
    <td> </td>
    <td> </td>
  </tr>
  <tr>
    <td>normiert</td>
    <td> </td>
    <td> </td>
  </tr>
</table>
<img src = 'a2-2.jpg'>

### Veränderung der Dynamik
Veränderungen der Dynamik akustischer Signale können z. B. zum Ausgleich von Lautstärkeunterschieden genutzt werden. Dazu wird das Signal in Zeitfenster eingeteilt, für die jeweils die maximal mögliche Verstärkung ermittelt wird. Diese kann durch Auswertung der benachbarten Zeitfenster ausgeglichen werden. Mit dem pro Zeitfenster ermittelten Faktor wird das Signal verstärkt und anschließend auf das Maximum normiert.

### <font color="#008000">Aufgabe 3</font>
Untersuchen Sie dieses Verfahren für unterschiedliche Signale, Zeitfensterlängen und Glättungsbereiche. Tragen Sie in der Tabelle **Tabelle Dynamikbearbeitung** die gewählten Parametergrößen ein und beschreiben Sie die Veränderung im Klang.

#### <font color="#FF0000">Eingabezelle 3</font>

In [ ]:
file = 'musik_dyn'
#file = 'instrumente_dyn'
#file = 'frau_clean_dyn'

#### Daten einlesen und darstellen

In [ ]:
data, fs = do.get_data(file)
Audio(data = data, rate = fs)

#### <font color="#FF0000">Eingabezelle 4</font>

In [ ]:
zf_len = 1000     # Zeitfenster [ms]
smooth_fac = 3   # Glättung über ... Zeitfenster

In [ ]:
data_dyn = do.get_gain(zf_len, fs, data, smooth_fac)
Audio(data = data_dyn, rate = fs)

In [ ]:
do.data_comp(data_dyn, data, fs)

##### Protokoll
#### Aufgabe 3  Dynamikbearbeitung
<table>
    <colgroup>
       <col span="1" style="width: 20%;">
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 10%;">
       <col span="1" style="width: 60%;">
    </colgroup>
  <tr>
    <th>Signal</th>
    <th>Zeitfensterlänge</th> 
    <th>Glättungsbereich</th>
    <th>Veränderung im Klang</th>
  </tr>
  <tr>
    <th>musik</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>musik</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>frau_clean</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>frau_clean</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>instrumente</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
  <tr>
    <th>instrumente</th>
    <th> </th> 
    <th> </th>
    <th> </th>
    <th> </th>
  </tr>
</table>

<img src = 'image/dynamic.jpg'>

Vor der Generierung des Protokolls in der nächsten Zelle speichern Sie bitte den aktuellen Stand mit `File -> Save and Checkpoint`.

In [ ]:
nb_in = 'DSVMI_part2.ipynb'
nb_out = 'Protokoll_part2.ipynb'
tp.protokoll(nb_in,nb_out)